In [11]:
import numpy as np
import matplotlib as plt
import math
import os
from pathlib import Path
import pandas as pd

In [34]:
base_dir = r'C:\Users\srini\Documents\GitHub\Paton-Lab--Summer-2024\Splinter' # Champalimaud PC dir
session = 0 # Change based on which trial you want
date = os.listdir(base_dir)[session]
path = os.path.join(base_dir, date)
os.chdir(path)

log_df = pd.read_csv('LogDf.csv')
session_df = pd.read_csv('SessionDf.csv')

log_data = log_df.to_dict(orient= 'dict')
session_data = session_df.to_dict(orient= 'dict')

spike_times = np.load('spiketimes.npy', allow_pickle=True)
NUM_NEURONS = spike_times.shape[0]
NUM_TRIALS = len(session_data['Unnamed: 0'])

In [36]:
print('Date: ', os.listdir(base_dir)[session])
print('Number Neurons: ', NUM_NEURONS)
print('Number Trials: ', NUM_TRIALS)
print('Log Keys: ', log_data.keys())
print('Session Keys: ', session_data.keys())

Date:  2016-07-27
Number Neurons:  19
Number Trials:  341
Log Keys:  dict_keys(['Unnamed: 0', 't', 'name', 'var', 'value'])
Session Keys:  dict_keys(['Unnamed: 0', 'chosen_side', 'chosen_interval', 'correct_side', 'interval_category', 'has_choice', 'this_interval', 'outcome', 'init_port', 'choice_rt', 't_on', 't_off', 't'])


In [37]:
def remove_nans(arr):
    return [x for x in arr if not math.isnan(x)]

for cell in range(NUM_NEURONS):
    spike_times[cell] = remove_nans(spike_times[cell])  

In [39]:
stim_onset_t = []
stim_offset_t = []
invalid_session_nums = []

for index in range(len(log_data['t'])):
    time = log_data['t'][index]
    name = log_data['name'][index]

    if name == 'STIM_ONSET_EVENT':
        stim_onset_t.append(time)